In [11]:
# A n'utiliser que si sur machine différente de GPU

!pip install gensim
!pip install scikit-learn
!pip install pandas
!pip install jupyter
!pip install nltk
!pip install matplotlib
!pip install spacy
!pip install wordcloud
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import gensim
from gensim import corpora
import gensim.downloader as api
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
from nltk.corpus import stopwords
import nltk
from pprint import pprint

In [50]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # optional but recommended
import spacy
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from wordcloud import WordCloud
from collections import Counter

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaktu\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kaktu\AppData\Roaming\nltk_data...


In [14]:
# pour Word2vec
wv = api.load('word2vec-google-news-300')

In [15]:
!python -m spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 0.8/400.7 MB 4.5 MB/s eta 0:01:30
     ---------------------------------------- 2.4/400.7 MB 6.0 MB/s eta 0:01:07
     ---------------------------------------- 3.9/400.7 MB 7.0 MB/s eta 0:00:57
      --------------------------------------- 5.5/400.7 MB 7.2 MB/s eta 0:00:55
      --------------------------------------- 7.6/400.7 MB 7.7 MB/s eta 0:00:52
      --------------------------------------- 9.2/400.7 MB 7.8 MB/s eta 0:00:51
     - ------------------------------------- 10.7/400.7 MB 7.8 MB/s eta 0:00:51
     - ------------------------------------- 12.8/400.7 MB 8.0 MB/s eta 0:00:49
     - ------------------------------------- 14.9/400.7 MB 8.1 MB/s eta 0:00:48
     - ------------------------------------- 16.3/400.7 MB 8.0 MB/s eta 0:00:48
     - ------------------------------------- 18.4


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------------------------- ------------ 269.5/400.7 MB 5.1 MB/s eta 0:00:26
     ------------------------- ------------ 269.5/400.7 MB 5.1 MB/s eta 0:00:26
     ------------------------- ------------ 269.7/400.7 MB 5.1 MB/s eta 0:00:26
     ------------------------- ------------ 270.3/400.7 MB 5.0 MB/s eta 0:00:26
     ------------------------- ------------ 270.5/400.7 MB 5.0 MB/s eta 0:00:27
     ------------------------- ------------ 270.8/400.7 MB 4.9 MB/s eta 0:00:27
     ------------------------- ------------ 271.1/400.7 MB 4.9 MB/s eta 0:00:27
     ------------------------- ------------ 271.6/400.7 MB 4.9 MB/s eta 0:00:27
     ------------------------- ------------ 271.8/400.7 MB 4.8 MB/s eta 0:00:27
     ------------------------- ------------ 272.4/400.7 MB 4.8 MB/s eta 0:00:27
     ------------------------- ------------ 272.9/400.7 MB 4.7 MB/s eta 0:00:28
     ------------------------- ------------ 273.4/400.7 MB 4.7 MB/s eta 0:00:28
     -------------------------- -------

In [97]:
def is_valid_word(w):
    """
        Vérifie si un mot ne contient pas de caractères spéciaux ni de chiffres
    """
    return (
        w.isalpha() and
        w.islower() and
        len(w) > 2 and
        "_" not in w
    )


In [98]:
def wordnet_synonyms(word, pos=wn.NOUN):
    syns = set()
    for synset in wn.synsets(word, pos=pos):
        for lemma in synset.lemmas():
            w = lemma.name().lower().replace('_', ' ')
            if w.isalpha():
                syns.add(w)
    return syns


In [102]:
def plus_proche_voisin(mot, forbid_e=False, inverse = False):
    """
    Retourne le mot le plus proche de 'mot' selon les vecteurs de mots (Word2Vec),
    avec options pour interdire les mots contenant 'e' ou sélectionner uniquement ceux
    contenant 'e'.
    
    Paramètres :
    - mot : str, mot de départ
    - forbid_e : bool, si True, exclut les mots contenant 'e'
    - inverse : bool, si True, garde uniquement les mots contenant 'e'
    
    Retourne :
    - str : mot le plus similaire selon la similarité cosinus
    """
    if mot not in wv:
        return mot

    mot_vecteur = wv[mot]
    candidats = []

    for elt in wordnet_synonyms(mot):
        if elt == mot:
            continue
        if forbid_e and 'e' in elt:
            continue
        if inverse and 'e' not in elt:
            continue
        if elt not in wv:
            continue

        cos = cosine_similarity(mot_vecteur, wv[elt])
        candidats.append((elt, cos))

    if not candidats:
        for elt in wv.index_to_key:
            if elt == mot:
                continue
            if forbid_e and 'e' in elt:
                continue
            if inverse and 'e' not in elt:
                continue
            if not is_valid_word(elt):
                continue
    
    
            cos = cosine_similarity(mot_vecteur, wv[elt])
            candidats.append((elt, cos))

        if not candidats:
            return mot

            

    candidats.sort(key=lambda x: x[1], reverse=True)
    return candidats[min(0, len(candidats)-1)][0]


In [99]:
from numpy.linalg import norm
def cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [100]:
def get_fruits(inverse=False):
    """
        Retourne une liste de mots liés aux fruits
        Paramètres :
        - inverse : bool, si True, ne garde que les mots contenant 'e'
                          si False, ne garde que les mots ne contenant pas 'e'

    """
    fruits_wv = wv.most_similar(positive=['fruit'], topn=20)
    fruits = []
    if inverse:
        for x in fruits_wv:
            if 'e' in  x[0] and is_valid_word(x[0]):
                fruits.append(x[0])
    else:
        for x in fruits_wv:
            if not 'e' in  x[0] and is_valid_word(x[0]):
                fruits.append(x[0])
    return fruits         

     

In [103]:
# sans e

import spacy
import random
from collections import Counter

nlp = spacy.load("en_core_web_lg")

with open(r"C:\université\tal\TP2-ETUDIANT\sport-en.txt", encoding="utf-8") as f:
    text = f.read()

doc = nlp(text)

new_tokens = []

fruits = get_fruits()


for token in doc:
    forbid_e = 'e' in token.text.lower()


    if not token.is_punct and forbid_e: 
        # forbid_e = 'e' in token.text.lower()
        
        remplacement = plus_proche_voisin(
                token.lemma_.lower(),
                forbid_e=forbid_e
            )
        

        # preserve capitalization
        if token.text[0].isupper():
            remplacement = remplacement.capitalize()
        if remplacement == token.text:
            if token.pos_ == "PROPN":
                remplacement = fruits[random.randint(0, len(fruits)-1)]
                new_tokens.append(remplacement)
                print(remplacement, end =  " ")
                continue
        print(remplacement, end =  " ")
        new_tokens.append(remplacement)
    else:
        print(token.text, end =  " ")
        new_tokens.append(token.text)

    

rewritten_text = spacy.tokens.Doc(nlp.vocab, words=new_tokens)

This US Man ’s National Squad ( USMNT ) was shuffling to work hard for its spot in this World Cup knockout point , attain a tough 1 - 0 win past Iran in a spunk - racking Group B match at Qatar 2022 . 

 Christian Pulisic ’s first - half goal was all that distinct this two position as this USMNT grasp this round of 16 for this first clip has 2014 . 

 This scrutiny was obviously on for Carlson mango ’s position in Qatar following this national squad ’s bankruptcy to qualify for this 2018 World Cup , but this US got this job ut with trio gritty group functioning . 


 

In [104]:
# avec e

import spacy
import random
from collections import Counter

nlp = spacy.load("en_core_web_lg")

with open(r"C:\université\tal\TP2-ETUDIANT\sport-en.txt", encoding="utf-8") as f:
    text = f.read()

doc = nlp(text)

new_tokens = []

fruits = get_fruits(inverse = True)


for token in doc:
    allow_l = 'e' not in token.text.lower()


    if not token.is_punct and allow_l: 
        
        remplacement = plus_proche_voisin(
                token.lemma_.lower(),
                forbid_e=forbid_e,
                inverse = True
            )
        

        # preserve capitalization
        if token.text[0].isupper():
            remplacement = remplacement.capitalize()
        if remplacement == token.text:
            if token.pos_ == "PROPN":
                remplacement = fruits[random.randint(0, len(fruits)-1)]
                new_tokens.append(remplacement)
                print(remplacement, end =  " ")
                continue
        print(remplacement, end =  " ")
        new_tokens.append(remplacement)
    else:
        print(token.text, end =  " ")
        new_tokens.append(token.text)

    

rewritten_text = spacy.tokens.Doc(nlp.vocab, words=new_tokens)

The America Men ’s Subject Team ( Usmnt ) beryllium made to employment harder forthe itself place inthe the Globe Demitasse stunner stage , earning a toughie single - zero victories over Persia inthe a nerve - wheel Committee Bel decider astatine Uae 2022 . 

 Athiest melon ’s beginning - quarter end beryllium these however separated the deuce sides ampere the Usmnt reached the beat of 16 forthe the beginning time since 2014 . 

 The pressure beryllium certainly onthe forthe Gregg Berhalter ’s side inthe Uae after the subject team ’s failure to qualifies forthe the 2018 Globe Demitasse , because the America getting the chore done withthe three grittier committee performances . 


 